In [1]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
import openpyxl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
import statistics
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()
from scipy.ndimage.filters import gaussian_filter1d
import warnings
warnings.filterwarnings("ignore")
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import sem
from scipy import stats
from scipy.signal import savgol_filter
import random
import os

C:\Users\AllenLi\AppData\Local\Temp\ipykernel_29776\2190129786.py:15: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [2]:
def addMissingTimesAndInterpolateDF(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Returns a dataframe with missing times added in and interpolated. 
    Prints out times that were added. \n
    DOES NOT MODIFY DataFrame IN PLACE. \n
    '''
    maxTime = int(10*max(df.index))
    minTime = int(10*min(df.index))
    allTimes = np.round(np.linspace(minTime/10, maxTime/10, num=maxTime-minTime+1), 1)
    unincludedTimes = set(allTimes).difference(set(df.index))
    df2 = df.copy()
    for t in unincludedTimes:
        df2.loc[t] = np.nan
    #print("Added", len(unincludedTimes), "times:", sorted(unincludedTimes))
    df2 = df2.sort_index()
    df2.index.name = 'Time(s)'
    df2.interpolate(inplace=True)
    return df2

# Load Data

In [3]:
#set iteration 
directory= "D:/Xiguang/sample_data/"
outputdirectory = "D:/Xiguang/sample_data/"
dimneeded = 2
repeats =5
for listname in ["WT"]:
    if listname == "WT":
        animal_nums = ["152","153","174","175","177","180","181"]
        
    for number in range(repeats):
        print(listname)
        print("itr",number)
        for an in animal_nums:
            it_num = 20
            
            #Read Lever Times
            time_lower_range = -15
            time_upper_range = 15
            tot_time = (time_upper_range - time_lower_range)*10 + 1

            pal = sns.color_palette('husl', 9)

            animal_num = an
            day_num = 'Day11'

            file_prefix = animal_num + '_' + day_num
            
            


            # Read all times
            standardized_path = os.path.join(directory,"z_score/"+ file_prefix + '_standardized.csv')
            #standardized_path = pd.read_csv('D2#52_Day11_standardized.csv')
            df_WD1 = pd.read_csv(standardized_path)
            df_WD1 = df_WD1.set_index('Time(s)')
            WD1 = addMissingTimesAndInterpolateDF(df_WD1)

            # Read ALP Timing
            alp_timing_path = os.path.join(directory, "press_timing/"+file_prefix + '_Lever_Press_Timing.csv')
            ALP = pd.read_csv(alp_timing_path)
            ALP_ori = ALP['Time(s)'].tolist()
            ALP = ALP['Time(s)'].tolist()

            #Read ILP Timing (To Select Baseline Data Outside of These Points)
            ilp_timing_path = os.path.join(directory, "press_timing/"+file_prefix + '_Inactive_Lever_Press_Timing.csv')
            ILP = pd.read_csv(ilp_timing_path)
            ILP_ori = ILP['Time(s)'].tolist()
            ILP = ILP['Time(s)'].tolist()

            # initializing K (20s)
            K = 10

            idx = 0
            while idx < len(ALP) - 1:

                # checking for difference
                if ALP[idx] + K > ALP[idx + 1]:

                    # deleting if K closer
                    del ALP[idx + 1]
                else:
                    idx += 1
            i=0
            while i < len(ALP):
                if ALP[i] + time_upper_range >= WD1.index[-1]: #If ALP + 20s exceeds the length of recording time, then delete ALP
                    del ALP[i]
                else:
                    i = i+1
            i=0
            while i < len(ALP):
                if ALP[i] + time_lower_range <= WD1.index[0]:
                    del ALP[i]
                else:
                    i = i+1    
            ALP_all_timing_WD1 = []
            ALP_temp = []

            for y in ALP:
                temp_list = np.around(np.arange((y + time_lower_range)*10, (y + time_upper_range)*10 + 1, 1)/10, decimals=1)[:tot_time]
                ALP_temp.append(temp_list)
            ALP_all_timing_WD1.append(ALP_temp)

            #Create Concatenated DataFrame of ALP Activity
            temp_concat = []
            ALP_separate_WD1 = []
            ALP_concat_WD1 = pd.DataFrame()

            for x in range(len(ALP)):
                temp_concat= WD1.loc[ALP_all_timing_WD1[0][x]]
                temp_concat = temp_concat.reset_index()
                temp_concat = temp_concat.drop(columns=['Time(s)'])
                ALP_separate_WD1.append(temp_concat)
                ALP_concat_WD1 = pd.concat([ALP_concat_WD1, temp_concat])

            LP_WIDTH = 10
            WIDTH_Modifier=10
            BL_WIDTH = 5
            while WIDTH_Modifier>= 5:
                createInterval = lambda y: np.arange((y - LP_WIDTH)*10, (y + LP_WIDTH)*10 + 1, 1)/10
                all_times = list(WD1.index.values)
                non_LP_WD1 = all_times 


                
                num_select = 2*len(ALP)
                to_select_times_filtered = np.array(non_LP_WD1)
                to_select_times_filtered = to_select_times_filtered[to_select_times_filtered < max(to_select_times_filtered) - time_upper_range]
                to_select_times_filtered = to_select_times_filtered[to_select_times_filtered > min(to_select_times_filtered) - time_lower_range]
                times = []
                max_len_times = []
                num_retries = 20
                # Run the random selection multiple times and pick the largest time array 
                while len(times) != num_select and num_retries > 0: 
                    for currCount in range(num_select):
                        if len(to_select_times_filtered) == 0: 
                            # The points selected don't space out by 20 well
                            break
                        time = np.random.choice(to_select_times_filtered, 1)[0]
                        to_select_times_filtered = [t for t in to_select_times_filtered if not(t - BL_WIDTH <= time <= t + BL_WIDTH)]
                        times.append(time)
                    # Check for larger time selection and keep the bigger one, reset to pick again
                    if len(times) > len(max_len_times):
                        max_len_times = times
                    num_retries -= 1
                    to_select_times_filtered = np.array(non_LP_WD1)
                    to_select_times_filtered = to_select_times_filtered[to_select_times_filtered < max(to_select_times_filtered) - time_upper_range]
                    to_select_times_filtered = to_select_times_filtered[to_select_times_filtered > min(to_select_times_filtered) - time_lower_range]
                    times = []
                times = max_len_times
                if len(times) == num_select:
                    break
                else:
                    #print("NOT ENOUGH RANDOM TIME POINTS! Selected", len(times), "when we want", num_select)
                    if WIDTH_Modifier >5:
                        WIDTH_Modifier = WIDTH_Modifier-1
                    else:
                        BL_WIDTH = BL_WIDTH-1
                    if BL_WIDTH == 0:
                        raise ValueError("BL_WIDTH = 0 for " + file_prefix) 
                # assert(not isOverlapping(times)) # Sanity check
            times.sort()

            random.shuffle(times)
            t = len(ALP)
            BL1 = times[:t]
            BL2 = times[t:]
            df_BL_1 = WD1.loc[BL1]
            BL1_timing = np.array(df_BL_1.index.values)
            BL1_timing = list(BL1_timing)

            BL1_all_timing = []
            

            
            BL_temp = []

            for y in BL1_timing:
                temp_list = np.around(np.arange(round((y + time_lower_range)*10), round((y + time_upper_range)*10 + 1), 1)/10, decimals=1)
                temp_list = [round(x, 1) for x in temp_list]
                BL_temp.append(temp_list)
            BL1_all_timing.append(BL_temp)


            #Create Concatenated DataFrame of BL Activity
            temp_concat = []
            BL1_separate = []
            BL1_concat = pd.DataFrame()

            for x in range(len(BL1_timing)):
                temp_concat= WD1.loc[BL1_all_timing[0][x]]
                temp_concat = temp_concat.reset_index()
                temp_concat = temp_concat.drop(columns=['Time(s)'])
                BL1_separate.append(temp_concat)
                BL1_concat = pd.concat([BL1_concat, temp_concat])

            by_row_index_1 = BL1_concat.groupby(BL1_concat.index)
            BL1_mean = by_row_index_1.mean()
            
            df_BL_2 = WD1.loc[BL2]
            BL2_timing = np.array(df_BL_2.index.values)
            BL2_timing = list(BL2_timing)
            BL2_all_timing = []
            BL_temp = []

            for y in BL2_timing:
                temp_list = np.around(np.arange(round((y + time_lower_range)*10), round((y + time_upper_range)*10 + 1), 1)/10, decimals=1)
                temp_list = [round(x, 1) for x in temp_list]
                BL_temp.append(temp_list)
            BL2_all_timing.append(BL_temp)


            #Create Concatenated DataFrame of BL Activity
            temp_concat = []
            BL2_separate = []
            BL2_concat = pd.DataFrame()

            for x in range(len(BL2_timing)):
                temp_concat= WD1.loc[BL2_all_timing[0][x]]
                temp_concat = temp_concat.reset_index()
                temp_concat = temp_concat.drop(columns=['Time(s)'])
                BL2_separate.append(temp_concat)
                BL2_concat = pd.concat([BL2_concat, temp_concat])

            by_row_index_2 = BL2_concat.groupby(BL2_concat.index)
            BL2_mean = by_row_index_2.mean()
            by_row_index_WD1 = ALP_concat_WD1.groupby(ALP_concat_WD1.index)
            WD1_ALP_mean = by_row_index_WD1.mean()
            df_concat_WD1 = pd.concat([WD1_ALP_mean,BL1_mean,BL2_mean])
            from sklearn.pipeline import Pipeline

            pipe = Pipeline([('scaler', StandardScaler()),('pca', PCA())])
            pipe.fit(df_concat_WD1) #only call the fit method
            pca = pipe.named_steps['pca']
            np.savetxt("D:/Xiguang/PCAALP2BL/" + animal_num + '_' + day_num + 'variance_explained_' + str(number) + '.csv', pca.explained_variance_  , delimiter=",")
            projected_BL1_trials = []
            projected_BL2_trials = []
            projected_ALP_trials = []
            
            
            for x in range(len(ALP)):
                # project every trial using the pca fit on averages
                proj_ALP_trial = pipe.transform(ALP_separate_WD1[x]).T
                projected_ALP_trials.append(proj_ALP_trial)
            # transform each trial
            for x in range(len(BL1)):
                # project every trial using the pca fit on averages
                proj_BL1_trial = pipe.transform(BL1_separate[x]).T
                projected_BL1_trials.append(proj_BL1_trial)



            # transform each trial
            for x in range(len(BL2)):
                # project every trial using the pca fit on averages
                proj_BL2_trial = pipe.transform(BL2_separate[x]).T
                projected_BL2_trials.append(proj_BL2_trial)

            projected_ALP_trials = np.array(projected_ALP_trials)
            projected_ALP_trials = projected_ALP_trials[:,:,0:tot_time]
            projected_BL1_trials = np.array(projected_BL1_trials)
            projected_BL1_trials = projected_BL1_trials[:,:,0:tot_time]
            projected_BL2_trials = np.array(projected_BL2_trials)
            projected_BL2_trials = projected_BL2_trials[:,:,0:tot_time]
            for pcwrite in range(dimneeded):
                np.savetxt(outputdirectory + animal_num + '_' + day_num + 'trials_BL1_PC'+str(pcwrite+1)+'_it' + str(number) + '.csv', projected_BL1_trials[:,pcwrite,:] , delimiter=",")
                np.savetxt(outputdirectory + animal_num + '_' + day_num + 'trials_BL2_PC'+str(pcwrite+1)+'_it' + str(number) + '.csv', projected_BL2_trials[:,pcwrite,:] , delimiter=",")
                np.savetxt(outputdirectory + animal_num + '_' + day_num + 'trials_ALP_PC'+str(pcwrite+1)+'_it' + str(number) + '.csv', projected_ALP_trials[:,pcwrite,:] , delimiter=",")
            

WT
itr 0
WT
itr 1
WT
itr 2
WT
itr 3
WT
itr 4
